This notebook uses the data from the pears project to collect individual catalogs of low mass and high mass major and minor pairs.

In [1]:
import h5py
import numpy as np
import matplotlib.pyplot as plt
import sys
import importlib
import pandas as pd

from utils_orbs.orb_paths import SetupPaths
import utils_orbs.readsubfHDF5Py3 as readSub
# from utils.read_group_cats import ReadCats
from utils_orbs.merger_trees import TraceMergerTree
from utils_orbs.readMergerHistoryFiles import FindHistory
from utils_orbs.vectorCorrection import vectorCorrection as vector

paths = SetupPaths()

imported h5py...
modules seem to have loaded?


In [2]:
plt.show();
plt.rcParams.update({'font.size':20,"xtick.direction":"in","ytick.direction":"in", 
                     "xtick.top":True, "ytick.right":True,"text.usetex":False,
                     "xtick.labelsize":18,"ytick.labelsize":18})

## Create data catalog 

get all pair types (minor low mass and minor high mass) ~ 
note: snapshot 48 is missing in the hydro ver of TNG100

In [3]:
# make functions to get data at the requested snapshot
def get_primmask(primstells, size):
    if size == "low":
        mask = (primstells > 0.01) & (primstells < 0.5)
    elif size == "high":
        mask = (primstells > 0.5) & (primstells < 10)
    return mask

def get_groupmask(groupmass, size):
    if size == "low":
        mask = (groupmass > 8) & (groupmass < 50)
    elif size == "high":
        mask = (groupmass > 100) & (groupmass < 650)
    return mask

'/xdisk/gbesla/katiechambe/orbs/data/'

In [15]:
def pull_pairs(snapshot, size, massratio):
    units_dict = {
                "Group ID":"Group Number in Subfind Catalogs", 
                "Group Mass":"Physical mass from Group_M_TopHat200 -- 1e10 Msun",
                "Group Radius":"Physical radius from Group_R_TopHat200 -- kpc",
                "Group Nsubs":"Number of subhalos in group",
                "Sub1 ID":  "Subhalo ID at current snapshot",
                "Sub2 ID":  "Subhalo ID at current snapshot",
                "Sub1 Mass": "Subhalo mass at current snapshot -- 1e10 Msun",
                "Sub2 Mass": "Subhalo mass at current snapshot -- 1e10 Msun",
                "Sub1 Stellar Mass": "Stellar masses from abundance matching -- 1e10 Msun",
                "Sub2 Stellar Mass": "Stellar masses from abundance matching -- 1e10 Msun",
                "Sub1 Pos": "Subhalo physical position in box x,y,z -- kpc",
                "Sub2 Pos": "Subhalo physical position in box x,y,z -- kpc",
                "Sub1 Vel": "Subhalo velocity in vx, vy, vz -- km/s",
                "Sub2 Vel": "Subhalo velocity in vx, vy, vz -- km/s",
                "Sub1 MassType": "Mass of bound particles - gas, DM, empty, tracers, stars, BHs -- in 1e10 Msun",
                "Sub2 MassType": "Mass of bound particles - gas, DM, empty, tracers, stars, BHs -- in 1e10 Msun",
                "Separation": "Physical separation between primary and secondary in kpc",
                "Comoving Separation":"Comoving separation between primary and secondary in ckpc",
                "RelVel": "Relative velocity between primary and secondary in km/s",
                "Stellar Mass Ratio": "Stellar mass ratio of secondary over primary", # in this case- it's by primary subhalo mass
                "Realization": "Stellar mass realization (0-1000)",
                "Sub1 BHMass": "Sum of the masses of all blackholes -- 1e10 Msun",
                "Sub2 BHMass": "Sum of the masses of all blackholes -- 1e10 Msun",
                "Sub1 BHMdot": "Instantaneous accretion rates of all blackholes -- 1e10 Msun / 0.978Gyr",
                "Sub2 BHMdot": "Instantaneous accretion rates of all blackholes -- 1e10 Msun / 0.978Gyr",
                "Sub1 SFR": "Sum of the individual SFRs of all gas cells in subhalo -- Msun / yr",
                "Sub2 SFR": "Sum of the individual SFRs of all gas cells in subhalo -- Msun / yr",
                "Sub1 SFRinRad": "Sum of SFRs of all gas cells within twice the stellar half mass radius -- Msun / yr",
                "Sub2 SFRinRad": "Sum of SFRs of all gas cells within twice the stellar half mass radius -- Msun / yr",
                "Sub1 GasMetallicity": "Mz/Mtot, where Z = any element above He within 2x stellar half mass radius -- unitless",
                "Sub2 GasMetallicity": "Mz/Mtot, where Z = any element above He within 2x stellar half mass radius -- unitless",
                "TripleFlag": "0 if no tertiary with mass ratio > 1:10 of secondary, 1 if large tertiary, 2 if other problem occurred"
                }
    
    snapdata = h5py.File(f"/xdisk/gbesla/katiechambe/pears/data/pairs/TNG_{snapshot}_10.hdf5","r")
    pairs = {}
    for key, val in snapdata['pairs']['hydro'].items():
        pairs[key]=np.array(val)
    redshift = snapdata["Header"].attrs["Redshift"]
    scale = 1/(1+redshift)
    snapdata.close()

    maskk  = pairs['Realization']==-1    

    medianpairs = {}
    for key in pairs.keys():
        medianpairs[key] = pairs[key][maskk]

    # get out the things that are in dwarf groups:
    group_mask = get_groupmask(medianpairs['Group Mass'],size)
    primary_mask = get_primmask(medianpairs['Sub1 Stellar Mass'], size)
    major_mask = (medianpairs['Sub2 Stellar Mass']/medianpairs['Sub1 Stellar Mass'] > 1/4)
    minor_mask = (medianpairs['Sub2 Stellar Mass']/medianpairs['Sub1 Stellar Mass'] > 1/10) & ~major_mask

    if massratio == "major":
        fullmask = group_mask&primary_mask&major_mask
    elif massratio == "minor":
        fullmask = group_mask&primary_mask&minor_mask

        # make data file
    f = h5py.File(f"{paths.path_data}collection/{size}mass_{massratio}_{snapshot}.hdf5", 'w')
    header_dict = {"Details":f"{size} mass {massratio} pairs at z={redshift:.1f}",
            "Snapshot":snapshot,
            "Scale":scale,
            "Redshift":redshift,
            "Simulation":"TNG100-1"}

    dset = f.create_group('/Header')
    for key in header_dict.keys():
        dset.attrs[key] = header_dict[key]

    for key, val in medianpairs.items():
        val = np.array(val)[fullmask]
        dset = f.create_dataset(f'/{key}', 
                                shape=val.shape,
                                dtype=val.dtype)
        dset.attrs[key] = units_dict[key]
        dset[:] = val

    f.close()
    print(f"Snapshot {snapshot} - total number of all pairs, median pairs, and {size} median {massratio} pairs: {len(pairs['Sub1 Stellar Mass'])}, {len(medianpairs['Sub1 Stellar Mass'])}, {len(medianpairs['Sub1 Stellar Mass'][fullmask])}")



In [16]:
pull_pairs(30, "high", "major")

Snapshot 30 - total number of all pairs, median pairs, and high median major pairs: 185244, 15437, 124


In [17]:
pull_pairs(30, "low", "major")

Snapshot 30 - total number of all pairs, median pairs, and low median major pairs: 185244, 15437, 1330


In [18]:
pull_pairs(30, "low", "minor")

Snapshot 30 - total number of all pairs, median pairs, and low median minor pairs: 185244, 15437, 1196


In [19]:
pull_pairs(30, "high", "minor")

Snapshot 30 - total number of all pairs, median pairs, and high median minor pairs: 185244, 15437, 129


In [23]:
for i in np.arange(0,100,1):
    if i == 48:
        continue
    pull_pairs(i, "low", "major")

Snapshot 0 - total number of all pairs, median pairs, and low median major pairs: 0, 0, 0
Snapshot 1 - total number of all pairs, median pairs, and low median major pairs: 0, 0, 0
Snapshot 2 - total number of all pairs, median pairs, and low median major pairs: 0, 0, 0
Snapshot 3 - total number of all pairs, median pairs, and low median major pairs: 0, 0, 0
Snapshot 4 - total number of all pairs, median pairs, and low median major pairs: 48, 4, 0
Snapshot 5 - total number of all pairs, median pairs, and low median major pairs: 108, 9, 1
Snapshot 6 - total number of all pairs, median pairs, and low median major pairs: 264, 22, 0
Snapshot 7 - total number of all pairs, median pairs, and low median major pairs: 600, 50, 4
Snapshot 8 - total number of all pairs, median pairs, and low median major pairs: 1260, 105, 10
Snapshot 9 - total number of all pairs, median pairs, and low median major pairs: 2220, 185, 12
Snapshot 10 - total number of all pairs, median pairs, and low median major pai